# Music Recommender System using Apache Spark and Python
**Estimated time: 8hrs**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this IPython file and do not clear any of the output you get from running your code.
* Upload this file onto moodle.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

## Necessary Package Imports

In [1]:
from pyspark.mllib.recommendation import *
import random
from operator import *

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [95]:
artistData = sc.textFile("/Users/Huangying/Courses/591BI/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/artist_data_small.txt")
artistAlias = sc.textFile("/Users/Huangying/Courses/591BI/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/artist_alias_small.txt")
userArtistData = sc.textFile("/Users/Huangying/Courses/591BI/01.Apache.Spark.Project-1.RecommenderSystems.FINAL/user_artist_data_small.txt")
#print userArtistData.take(1)[0]
#artistAlias.collect()

[u'1027859\t1252408',
 u'1017615\t668',
 u'6745885\t1268522',
 u'1018110\t1018110',
 u'1014609\t1014609',
 u'6713071\t2976',
 u'1014175\t1014175',
 u'1008798\t1008798',
 u'1013851\t1013851',
 u'6696814\t1030672',
 u'1036747\t1239516',
 u'1278781\t1021980',
 u'2035175\t1007565',
 u'1327067\t1308328',
 u'2006482\t1140837',
 u'1314530\t1237371',
 u'1160800\t1345290',
 u'1255401\t1055061',
 u'1307351\t1055061',
 u'1234249\t1005225',
 u'6622310\t1094137',
 u'1261919\t6977528',
 u'2103190\t1002909',
 u'9929875\t1009048',
 u'2118737\t1011363',
 u'9929864\t1000699',
 u'6666813\t1305683',
 u'1172822\t1127113',
 u'2026635\t1001597',
 u'6726078\t1018408',
 u'1039896\t1277013',
 u'1239168\t1266817',
 u'6819291\t1277876',
 u'2030690\t2060894',
 u'6786886\t166',
 u'1051692\t1307569',
 u'1239193\t1012079',
 u'1291581\t78',
 u'6642817\t1010969',
 u'1293171\t1007614',
 u'1070350\t1034635',
 u'6603691\t1279932',
 u'1027851\t1063053',
 u'2060513\t2029258',
 u'1277348\t668',
 u'1253023\t1033862',
 u'10028

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [150]:
def parser(s):
    temp = s.split(" ")
    list=[]
    list.append(int(temp[0]))
    list.append(int(temp[1]))
    list.append(int(temp[2]))
    return list
def transform(x):
    l = list(x[1])
    counts = []
    for i in range(0,len(l)):
        play = l[i][2]
        counts.append(play)
    total = sum(counts)
    avg = total/len(l)
    return (x[0],total,avg)  
temp = userArtistData.map(parser).groupBy(lambda x: x[0]).map(lambda x: (x[0],list(x[1]))).map(transform)
#print temp.collect()
top = temp.takeOrdered(3, key=lambda x: -x[1])
for i in range(0,len(top)):
    print "User " + str(top[i][0]) +" has a total play count of "+ str(top[i][1]) +" and a mean play count of "+ str(top[i][2]) +"."


User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [112]:
def parser(s):
    temp = s.split(" ")
    return (int(temp[0]),int(temp[1]),int(temp[2]))
temp = userArtistData.map(parser)
trainData,validationData,testData = temp.randomSplit([4,4,2], 13)
print trainData.take(3)
print validationData.take(3)
print testData.take(3)
print trainData.count()
print validationData.count()
print testData.count()
trainData.cache()
validationData.cache()
testData.cache()

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031


PythonRDD[1225] at RDD at PythonRDD.scala:43

## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [ ]:
def modelEval(model,validation):
    Count = validation.count()
    temp = validatian.map(lambda x: (x[0], x[1])
    result = model.predictAll(temp).map(lambda x: ((x[0], x[1]), x[2]))
    result.takeOrdered(Count, key=lambda x: -x[1])
                        
    return 0

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.090431
The model score for rank 10 is 0.095294
The model score for rank 20 is 0.090248
```

Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.

In [174]:
#trainData = trainData.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
#print validationData.map(lambda x: (x[0], x[1])).count()
predictions = bestModel.predictAll(validationData.map(lambda x: (x[0], x[1]))
).map(lambda x: (x[0], x[1], x[2])).groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1])))
validations = validationData.groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1])))
print predictions.count()
print validations.count()
#print result.collect()
#temp = result.groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1])))    
#temp.map(lambda x: (x[0],sorted(x[1],key=itemgetter(2),reverse=True)))
def match(l1,l2):
    length = len(l1)
    count = 0
    for i in range(0,length):
        e = l1[i]
        if (e in l2):
            count += 1 
#result.takeOrdered(testCount, key=lambda x: -x[1])
#modelEval(bestModel, testData)

19633
7653
50


## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
```